In [1]:
from rdkit import Chem
from rdkit.Chem import Descriptors

def calculate_descriptors(smiles_string):
    # Convert the SMILES string to a molecule object
    molecule = Chem.MolFromSmiles(smiles_string)

    # Calculate all available descriptors
    descriptors = {}
    for descriptor_name, descriptor_fn in Descriptors.descList:
        try:
            descriptors[descriptor_name] = descriptor_fn(molecule)
        except:
            descriptors[descriptor_name] = None

    return descriptors

In [2]:
smiles_string = "CC(=O)OC1=CC=CC=C1C(=O)O"
descriptors = calculate_descriptors(smiles_string)

In [3]:
descriptors.values()

dict_values([10.611948223733938, 10.611948223733938, 0.01601851851851821, -1.1140277777777776, 0.5501217966938848, 9.307692307692308, 180.15899999999996, 172.09499999999997, 180.042258736, 68, 0, 0.33900378687731025, -0.4775395271554559, 0.4775395271554559, 0.33900378687731025, 1.3076923076923077, 1.9230769230769231, 2.4615384615384617, 16.53611122125433, 10.182282381035343, 2.104306980957856, -2.0311320919470135, 2.1698345568128055, -2.063000061964297, 5.913065796110142, -0.13141434244030561, 2.0325369160995868, 3.0435273546341013, 343.2228677267164, 9.844934982691242, 6.9813595436500515, 6.9813595436500515, 6.109060905280622, 3.6174536478673316, 3.6174536478673316, 2.3949556783206725, 2.3949556783206725, 1.3711546649445037, 1.3711546649445037, 0.8871712192374142, 0.8871712192374142, -1.8399999999999999, 729.6807528797516, 9.249605734767023, 3.7092512583454584, 2.297415032519928, 74.75705264447721, 9.843390348640755, 11.3129633249809, 0.0, 0.0, 0.0, 11.938610575903699, 4.7945371840718

In [4]:
len(descriptors.values())

210

In [8]:
import json, os
from tqdm import tqdm
def create_SMILES_metadata(target_output_path):
    with open(os.path.join("molytica_m", "data_tools", "molecule_id_mappings", "id_to_smiles.json"), 'r') as f:
        id_to_smiles = json.load(f)
    
    for smiles in tqdm(id_to_smiles.values(), desc="Creating SMILES metadata"):
        descriptors = calculate_descriptors(smiles)
        print(descriptors)

In [9]:
create_SMILES_metadata("test")

FileNotFoundError: [Errno 2] No such file or directory: 'molytica_m/data_tools/molecule_id_mappings/id_to_smiles.json'